In [1]:
import os
import pickle
import numpy as np
import torch

from spenderq import load_model
from spenderq import util as U
from spenderq import lyalpha as LyA

In [2]:
from astropy.table import Table, vstack

In [3]:
# --- plotting --- 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False
%matplotlib inline

In [4]:
redshifts, tids = [], []
z_absorbers = []
for ibatch in range(85): 
    # load batch
    #with open('/tigress/chhahn/spender_qso/train/edr.lfsc.rsnr.sa1p1sb0p8.i%i_%i.pkl' % (niter, ibatch), "rb") as f:
    with open('/tigress/chhahn/spender_qso/train/DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        _, _, z, target_id, _, _ = pickle.load(f)
        
    # read absorbers
    i_absorb, z_absorb = [], [] 
    with open('/tigress/chhahn/spender_qso/absorbers/MgII.DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        out = pickle.load(f)
        i_absorb.append(np.concatenate([np.repeat(_o[0], len(_o[1])) for _o in out]))
        z_absorb.append([_o[1] for _o in out])
    i_absorb = np.concatenate(np.array(i_absorb))
    z_absorb = np.concatenate(z_absorb[0])
    z_absorbers.append(z_absorb)
    
    tids.append(np.array(target_id)[i_absorb])
    redshifts.append(np.array(z)[i_absorb])

tids = np.concatenate(tids)
redshifts = np.concatenate(redshifts)
z_absorbers = np.concatenate(z_absorbers)

In [6]:
mgii_absorbers = Table()
mgii_absorbers['ABSORBER'] = np.repeat('MgII', len(tids)).astype("<U24")
mgii_absorbers['TARGETID'] = tids
mgii_absorbers['Z_ABSORBER'] = z_absorbers
mgii_absorbers['Z_QSO'] = redshifts

In [7]:
print('%i MgII absorbers' % len(mgii_absorbers))

5709 MgII absorbers


In [8]:
redshifts, tids = [], []
z_absorbers = []
for ibatch in range(85): 
    # load batch
    #with open('/tigress/chhahn/spender_qso/train/edr.lfsc.rsnr.sa1p1sb0p8.i%i_%i.pkl' % (niter, ibatch), "rb") as f:
    with open('/tigress/chhahn/spender_qso/train/DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        _, _, z, target_id, _, _ = pickle.load(f)
    

    # read absorbers
    i_absorb, z_absorb = [], [] 
    with open('/tigress/chhahn/spender_qso/absorbers/CIV.DESI.edr.qso_highz_%i.pkl' % ibatch, 'rb') as f: 
        out = pickle.load(f)
        i_absorb.append(np.concatenate([np.repeat(_o[0], len(_o[1])) for _o in out]))
        z_absorb.append([_o[1] for _o in out])
    i_absorb = np.concatenate(np.array(i_absorb))
    z_absorb = np.concatenate(z_absorb[0])
    z_absorbers.append(z_absorb)

    tids.append(np.array(target_id)[i_absorb])
    redshifts.append(np.array(z)[i_absorb])

tids = np.concatenate(tids)
redshifts = np.concatenate(redshifts)
z_absorbers = np.concatenate(z_absorbers)

In [9]:
civ_absorbers = Table()
civ_absorbers['ABSORBER'] = np.repeat('CIV', len(tids)).astype("<U24")
civ_absorbers['TARGETID'] = tids
civ_absorbers['Z_ABSORBER'] = z_absorbers
civ_absorbers['Z_QSO'] = redshifts

In [10]:
print(len(civ_absorbers))

8018


# identify unique absorbers

In [11]:
for i in range(len(mgii_absorbers)): 
    tid, zabs = np.array(mgii_absorbers['TARGETID'])[i], np.array(mgii_absorbers['Z_ABSORBER'])[i]
    same_abs = (civ_absorbers['TARGETID'] == tid) & (np.abs(civ_absorbers['Z_ABSORBER'] - zabs) < 0.01)
    if np.sum(same_abs) == 0: continue
    
    mgii_absorbers['ABSORBER'][i] = 'MgII, CIV'
    civ_absorbers['ABSORBER'][same_abs] = ''

In [12]:
all_absorbers = vstack([mgii_absorbers, civ_absorbers[civ_absorbers['ABSORBER'] != '']])

In [15]:
all_absorbers[:100]

ABSORBER,TARGETID,Z_ABSORBER,Z_QSO
str24,int64,float64,float32
MgII,39628209360211028,1.740926369650913,2.3799982
MgII,39628226602996021,1.0086162395195515,2.1355138
MgII,39628249520673336,0.9511188945984902,2.2397785
MgII,39628249520673336,1.3911152392609616,2.2397785
MgII,39628277920302253,1.4133475171589256,2.5400681
MgII,39628289223953558,1.8906805735101107,3.4958878
MgII,39628289194594224,1.4379488988958886,2.6758394
MgII,39628294819155907,0.9529940237701774,2.267615
MgII,39628300452104833,0.7890041146983442,2.139522


In [16]:
all_absorbers.write('/tigress/chhahn/spender_qso/absorbers/DESI.edr.qso_highz.absorbers.hdf5', overwrite=True)

/home/chhahn/.conda/envs/gqp/lib/python3.11/site-packages/astropy/io/misc/hdf5.py:281: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  warnings.warn(
